In [ ]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache"

rc_dataset = load_dataset(
    "doc2dial",
    name="doc2dial_rc",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

In [4]:
import pandas as pd

rc_dataset_df = pd.DataFrame(data=rc_dataset)
rc_dataset_df.head()

,id,title,context,question,answers,domain
0,9f44c1539efe6f7e79b02eb1b413aa43_1,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,"user:Hello, I forgot o update my address, can ...",{'text': ['you must report a change of address...,dmv
1,9f44c1539efe6f7e79b02eb1b413aa43_3,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,user:Can I do my DMV transactions online? \tag...,"{'text': ['Sign up or log into MyDMV [6 ] '], ...",dmv
2,9f44c1539efe6f7e79b02eb1b413aa43_5,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,"user:Thanks, and in case I forget to bring all...",{'text': ['About ten percent of customers visi...,dmv
3,9f44c1539efe6f7e79b02eb1b413aa43_7,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,"user:Ok, and can you tell me again where shoul...",{'text': ['you must report a change of address...,dmv
4,9f44c1539efe6f7e79b02eb1b413aa43_9,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,user:Can you tell me more about Traffic points...,{'text': ['DMV maintains a point system to tra...,dmv


In [5]:
rc_dataset_df.describe()

,id,title,context,question,answers,domain
count,20431,20431,20431,20431,20431,20431
unique,20431,415,402,20257,6997,4
top,9f44c1539efe6f7e79b02eb1b413aa43_1,Benefits Planner | Social Security Administrat...,\n\nSocial Security Credits \nYou must earn at...,user:hi,{'text': ['You must earn at least 40 Social Se...,dmv
freq,1,101,181,11,41,5909


In [15]:
manual_doc_id = 'Learn about personalized plates#3_0'

manual_doc = rc_dataset.filter(lambda doc: doc['title'] == manual_doc_id)
manual_doc['question'][:5]

Loading cached processed dataset at ./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-2986451dff9bb180.arrow


['user:Can picture plates be personalized?',
 'user:How much do personalized plates cost? \tuser:For personalized plate availability and pricing for the picture plate you are interested in you can browse our collection of picture plates online \tagent:How much does that cost \tagent:Yes, some picture plates can be personalized for an additional fee in addition to your picture plate and regular registration fee \tuser:Can picture plates be personalized?',
 'user:Can I use personalized plates on any type of vehicle? \tagent:For Passenger and Commercial Plates the Initial Cost = $ 60.00Annual Fee = $ 31.25 \tuser:How much do personalized plates cost? \tuser:For personalized plate availability and pricing for the picture plate you are interested in you can browse our collection of picture plates online \tagent:How much does that cost \tagent:Yes, some picture plates can be personalized for an additional fee in addition to your picture plate and regular registration fee \tuser:Can picture p

# Compare to dialogue dataset

In [ ]:
split = "train"
cache_dir = "./data_cache_rc_notebook"

#User's turn: utterance= question, reference=grounding document span_id, can be empty, "precondition"/"solution" are the
#actual grounding spans
#Gold label for grounding
dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

In [8]:
# import pandas as pd

dialogue_full_df = pd.DataFrame(data=dialogue_dataset)

In [10]:
doc_id = 'Co-sign Your Spouse\'s Income-Driven Repayment Plan Application | Federal Student Aid#1_0'
dialogues_for_doc = dialogue_full_df.loc[dialogue_full_df['doc_id'] == doc_id]

for dialogue_index, turns in enumerate(dialogues_for_doc['turns']):
    dialid = dialogues_for_doc.iloc[dialogue_index]['dial_id']
    # print(dialogue_index)
    # print(dialid)

In [11]:
#Code to get the dialogues for a doc id from the dialog set to do some manual inspections
dialogue_eval_df = pd.DataFrame(columns=['doc_id', 'dial_id', 'rc_id', 'turn_id', 'user_utterance', 'user_sp_ids',
                                         'agent_sp_ids'])

#using df as dataset is way too slow to do this
dialogues_for_doc = dialogue_full_df.loc[dialogue_full_df['doc_id'] == doc_id]

#resetting all variables
user_turn_index = 0
user_utterance = ''
user_sps = []
turn_id = ''
agent_sps = []
role = ''
dial_id = ''
rc_id = ''
first_user = True
#for all list of turns in each dialogue
for dialogue_index, turns in enumerate(dialogues_for_doc['turns']):

    #for each turn in a dialogue
    for turn in turns:
        role = turn['role']

        # for the very first user role only
        if role == 'user' and first_user == True:
            turn_id = turn['turn_id']
            user_utterance =turn['utterance']
            user_sps = [ref['sp_id'] for ref in turn['references']]
            dial_id = dialogues_for_doc.iloc[dialogue_index]['dial_id']
            rc_id = dial_id + '_'+ str(turn_id)
            first_user = False
        elif role == 'user' and first_user == False:
            # we've come to the next user turn
            #write previous user's row before overwriting it with this users data
            dialogue_eval_df.loc[user_turn_index] = [doc_id, dial_id, rc_id, turn_id, user_utterance, user_sps, agent_sps]
            #remember this user's data
            user_utterance = turn['utterance']
            turn_id = turn['turn_id']
            user_sps = [ref['sp_id'] for ref in turn['references']]
            dial_id = dialogues_for_doc.iloc[dialogue_index]['dial_id']
            rc_id = dial_id + '_'+ str(turn_id)
            #reset for next row
            agent_sps = []
            user_turn_index += 1
        elif role == 'agent':
            #gather agent spans but don't write the row just yet
            agent_sps.extend([ref['sp_id'] for ref in turn['references']])

#write the last line to the dataframe if the last dialogue's turn ended with an agent
if role == 'agent':
    dialogue_eval_df.loc[user_turn_index] = [doc_id, dial_id, rc_id, turn_id, user_utterance, user_sps, agent_sps]


dialogue_eval_df.head()

,doc_id,dial_id,rc_id,turn_id,user_utterance,user_sp_ids,agent_sp_ids
0,Co-sign Your Spouse's Income-Driven Repayment ...,31e96f4f7c6325a6cd3875fd511c1d44,31e96f4f7c6325a6cd3875fd511c1d44_1,1,II'm going to be co-signing a federal aid loan...,[1],"[2, 3]"
1,Co-sign Your Spouse's Income-Driven Repayment ...,31e96f4f7c6325a6cd3875fd511c1d44,31e96f4f7c6325a6cd3875fd511c1d44_4,4,No. I'm not sure how it works.,[3],[]
2,Co-sign Your Spouse's Income-Driven Repayment ...,31e96f4f7c6325a6cd3875fd511c1d44,31e96f4f7c6325a6cd3875fd511c1d44_5,5,Does it matter that the Income-Driven Repaymen...,[12],[13]
3,Co-sign Your Spouse's Income-Driven Repayment ...,31e96f4f7c6325a6cd3875fd511c1d44,31e96f4f7c6325a6cd3875fd511c1d44_7,7,Who all can co-sign for this?,[16],[17]
4,Co-sign Your Spouse's Income-Driven Repayment ...,31e96f4f7c6325a6cd3875fd511c1d44,31e96f4f7c6325a6cd3875fd511c1d44_9,9,Do I need a Financial Information Note when I ...,[26],[26]


In [25]:
#Check if I can find any dialogue with its dial id in the rc document
def get_rc_question(df_in_rc):
    rc_question = df_in_rc['question']
    return rc_question.values[0]

def get_dialogue_question(id):
    user_utterance = dialogue_eval_df.loc[dialogue_eval_df['rc_id']== id]['user_utterance']
    return user_utterance.values[0]

ids_in_rc = []
empty_ids = []
user_utterance_not_in_rc = []
user_utterance_in_rc = []
rc_user_questions = []

for id in dialogue_eval_df['rc_id']:
    df_in_rc = rc_dataset_df.loc[rc_dataset_df['id'] == id]
    if not df_in_rc.empty:
        ids_in_rc.append(id)
        rc_question = get_rc_question(df_in_rc)
        user_utterance = get_dialogue_question(id)
        user_utterance_in_rc.append(user_utterance)
        rc_user_questions.append(rc_question)
    else:
        empty_ids.append(id)
        user_utterance = get_dialogue_question(id)
        user_utterance_not_in_rc.append(user_utterance)

example_dialogue = list(dialogue_eval_df['doc_id'])[0]
print(f'For the example dialogue: {example_dialogue}:\n')
print(f'Total number of question in dialogue: {dialogue_eval_df.shape[0]}')
print(f'Total number of question also in RC question for that dialogue: {len(ids_in_rc)}')
print(f'Total number of question not in RC question for that dialogue: {len(empty_ids)}\n')
print(f'Dialogue questions not in rc questions: {user_utterance_not_in_rc}\n')
print(f'Some Dialogue questions that are in rc question: {user_utterance_in_rc[:5]}')

For the example dialogue: Co-sign Your Spouse's Income-Driven Repayment Plan Application | Federal Student Aid#1_0:

Total number of question in dialogue: 48
Total number of question also in RC question for that dialogue: 40
Total number of question not in RC question for that dialogue: 8

Dialogue questions not in rc questions: ["No. I'm not sure how it works.", "Yes, I understand that I am not obligated to pay back the loan as a co-signer on the borrower's IDR application.", 'Yes, that was explained to me.', 'yes', 'no', "Yes, I'm concerned that I may be obligated to repay the loan by signing this application", 'no, but I might be ', 'Yes, I have read that I am not responsible to pay back the loan as a co-signer on my spouses IDR.']

Some Dialogue questions that are in rc question: ["II'm going to be co-signing a federal aid loan for my wife and I need some more information about how it works.", 'Does it matter that the Income-Driven Repayment Plan Request was done as part of a Direc